In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from matplotlib import rcParams
import requests

В этом блокноте реализван парсин биржевых цен на выбранную категорию по дням за последние 10 лет, при помощи yahoo finance api. 
(бесплатная верси) В данном случае мы парсим только стоимсть доллара к рублю.

https://rapidapi.com/blog/yahoo-finance-api-python/

In [40]:
def parce_the_data(host, key, interval, name, range_of_data, region):
    url = "https://yh-finance.p.rapidapi.com/stock/v2/get-chart"

    querystring = {"interval":interval,"symbol":name,"range":range_of_data,"region":region}

    headers = {
        'x-rapidapi-host': host,
        'x-rapidapi-key': key
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    return response.json()
  
def parseTimestamp(inputdata):
    timestamplist = []

    timestamplist.extend(inputdata["chart"]["result"][0]["timestamp"])
    timestamplist.extend(inputdata["chart"]["result"][0]["timestamp"])

    calendertime = []

    for ts in timestamplist:
        dt = datetime.fromtimestamp(ts)
        calendertime.append(dt.strftime("%m/%d/%Y"))
        
    return calendertime

def parseValues(inputdata):
    valueList = []
    valueList.extend(inputdata["chart"]["result"][0]["indicators"]["quote"][0]["open"])
    valueList.extend(inputdata["chart"]["result"][0]["indicators"]["quote"][0]["close"])

    return valueList

def attachEvents(inputdata):
    eventlist = []
    for i in range(0,len(inputdata["chart"]["result"][0]["timestamp"])):
        eventlist.append("open")  

    for i in range(0,len(inputdata["chart"]["result"][0]["timestamp"])):
        eventlist.append("close")

    return eventlist

def make_data_frame_with_history(host, key, interval, name, range_of_data, region):
    response = parce_the_data(host, key, interval, name, range_of_data, region)
    
    if (None != inputdata): 

        inputdata["Timestamp"] = parseTimestamp(response)

        inputdata["Values"] = parseValues(response)

        inputdata["Events"] = attachEvents(response)

        df= pd.DataFrame(inputdata)
        df = df[df['Events']=='open']
        df = df.drop('Events',axis=1)
        df = df.rename(columns = {'Values':name} )
        
        return df
            

In [41]:
#используя эту функцию мы получаем датафрейм с ежедневной ценой выбранной позиции на момент открытия торгов
#каждый день за последние 10 лет

#в данном случае мы парсим стоимость доллара к рублю

host = "yh-finance.p.rapidapi.com"
key = #мой апи ключ

df = make_data_frame_with_history(host, key, "1d", 'RUB=X', "10y", "US")

In [42]:
df

,Timestamp,RUB=X
0,03/05/2012,29.330200
1,03/06/2012,29.338200
2,03/07/2012,29.632200
3,03/08/2012,29.609200
4,03/09/2012,29.341200
...,...,...
2605,02/28/2022,83.865799
2606,03/01/2022,104.973503
2607,03/02/2022,108.479500
2608,03/03/2022,102.714996


In [43]:
df.to_csv('RUB=X.csv', index= False)